# Attachment Protocol Example -Bob


In [21]:
%autoawait
import time
import asyncio
import base64

IPython autoawait is `on`, and set to use `asyncio`


###  Intialise Controller

In [22]:
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

###  Check if active connection exists

In [23]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


Results :  [{'their_label': 'Alice', 'invitation_key': 'HAmnhn9ytFXjBMu927DFRjgStMJ7D33vRhHdkhzWi9Fe', 'routing_state': 'none', 'created_at': '2020-09-20 17:18:55.245042Z', 'their_did': '18mLaPQ24aZYcwtihn3hN', 'updated_at': '2020-09-20 17:28:26.243420Z', 'connection_id': '65401d5d-70e2-4d61-81fc-2e11c1bd3800', 'accept': 'manual', 'my_did': 'U6csGxrgMJYGUx2wBK6vWM', 'initiator': 'self', 'state': 'active', 'invitation_mode': 'once'}]
Connection : {'their_label': 'Alice', 'invitation_key': 'HAmnhn9ytFXjBMu927DFRjgStMJ7D33vRhHdkhzWi9Fe', 'routing_state': 'none', 'created_at': '2020-09-20 17:18:55.245042Z', 'their_did': '18mLaPQ24aZYcwtihn3hN', 'updated_at': '2020-09-20 17:28:26.243420Z', 'connection_id': '65401d5d-70e2-4d61-81fc-2e11c1bd3800', 'accept': 'manual', 'my_did': 'U6csGxrgMJYGUx2wBK6vWM', 'initiator': 'self', 'state': 'active', 'invitation_mode': 'once'}
Active Connection ID :  65401d5d-70e2-4d61-81fc-2e11c1bd3800


### Instantiate Listener

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In this instance the handler decodes the sent attachment and stores it. 

In [19]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    base64_bytes = data.encode('ascii')
    text = base64.b64decode(base64_bytes)
    file_content = text.decode('ascii')
    filepath = f"received_files/{filename}"
    f = open(filepath,"w+")
    f.write(file_content)
    f.close()
    
#     data = payload['content'] 
#     asyncio.get_event_loop().create_task(om_controller.messaging.send_message(connection_id, " Received!"))
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(attach_controller.listen_webhooks())
attach_controller.register_listeners([attach_listener], defaults=True)


In this instance the handler decodes the sent image and stores it in the received_files folder. 

In [24]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    file_content = base64.b64decode(data)
#     fh = open("image_received.png", "wb")
#     fh.write(text)
#     fh.close()
    filepath = f"received_files/{filename}"
    f = open(filepath,'wb')
    f.write(file_content)
    f.close()
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(attach_controller.listen_webhooks())
attach_controller.register_listeners([attach_listener], defaults=True)

Handle Attachment {'connection_id': '65401d5d-70e2-4d61-81fc-2e11c1bd3800', 'message_id': '37734586-2029-404b-b08c-7698818a1908', 'attachment_message': 'Here is the data you wanted', 'content': 'VGhpcyBpcyBhIHRlc3QgZmlsZSB0byBhdHRhY2gu', 'filename': 'test_file.txt', 'state': 'received'} 65401d5d-70e2-4d61-81fc-2e11c1bd3800
Attachment Received


In [62]:
response = await attach_controller.protocol.send_attachment(connection_id,data)

print(response)


NameError: name 'data' is not defined


# End of Tutorial

Be sure to terminate the controller so you can run another tutorial.


In [20]:
response = await attach_controller.terminate()
print(response)


None
